In [1]:
# Data params:
n_genes_list = [2000, 3000, 6000]
min_counts_list = [5, 10, 20]
# Training params:
batch_size_list = [128, 500, 10000]
nIter_list = [100, 500, 1000]
training_params = [(64, 1), (64, 10), (250, 200), (500, 300),
                   (1000, 400), (2000, 500), (5000, 500), (128, 200), (250, 400), (500, 600),
                   (1000, 800), (2000, 1000), (5000, 1000)]
# Model params:
mod_list = [1, 1.131, 0.8695]
v_ag_list=[{"alpha": 9.0, "beta": 3.0}, {"alpha": 12.0, "beta": 3.0}]
Tmax_list=[{"mean": 500., "sd": 150.}, {"mean": 250., "sd": 100.}]
import scvelo as scv
import scanpy as sc
import cell2fate as c2f
import pickle as pickle
from eval_utils import cross_boundary_correctness
from datetime import datetime
import pandas as pd
import numpy as np
from os.path import exists
import matplotlib.pyplot as plt
import torch
import unitvelo as utv
method = 'cell2fate'
data_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_datasets/'
save_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/'
dataset = 'Pancreas_with_cc'
n_genes_list = [2000]
n_genes = 2000
for i in range(len(training_params)):
            print(i)
            if not exists(save_dir + dataset + 'training_paramsLR005' + str(i) + '_CBDC_scores.csv'):
                adata = sc.read_h5ad(data_dir + dataset + '/' + dataset + '_anndata.h5ad')
                adata = c2f.utils.get_training_data(adata, cells_per_cluster = 10**5, cluster_column = 'clusters',
                                                remove_clusters = [], min_shared_counts = 10, n_var_genes= n_genes)
                c2f.Cell2fate_DynamicalModel.setup_anndata(adata, spliced_label='spliced', unspliced_label='unspliced')    
                n_modules = int(np.round(c2f.utils.get_max_modules(adata)*mod_list[0]))
                mod = c2f.Cell2fate_DynamicalModel(adata,
                                                   n_modules = n_modules,
                                                   stochastic_v_ag_hyp_prior= v_ag_list[0],
                                                   Tmax_prior= Tmax_list[0])   
                mod.train(max_epochs = training_params[i][1],
                          batch_size = training_params[i][0],
                          lr = 0.1)
                adata = mod.export_posterior(adata)
                fig, ax = plt.subplots(1,2, figsize = (15, 5))
                sc.pl.umap(adata, color = ['Time (hours)'], legend_loc = 'right margin',
                            size = 200, color_map = 'inferno', ncols = 2, show = False, ax = ax[0])
                sc.pl.umap(adata, color = ['Time Uncertainty (sd)'], legend_loc = 'right margin',
                            size = 200, color_map = 'inferno', ncols = 2, show = False, ax = ax[1])
                plt.savefig(save_dir + 'c2f_plots/' + dataset + 'training_params' + str(i) + '_UMAP_Time.svg')
                mod.compute_and_plot_total_velocity(adata, save = False, delete = False)
                # Calculate performance metrics:
                file = open(data_dir + dataset + '/' + dataset + '_groundTruth.pickle' ,'rb')
                ground_truth = pickle.load(file)
                metrics = utv.evaluate(adata, ground_truth, 'clusters', 'Velocity')
                if exists(save_dir + dataset + 'training_paramsLR005' + str(i) + '_CBDC_scores.csv'):
                    tab = pd.read_csv(save_dir + dataset + 'training_paramsLR005' + str(i) + '_CBDC_scores.csv', index_col = 0)
                else:
                    tab = pd.DataFrame(columns = list(metrics['Cross-Boundary Direction Correctness (A->B)'].keys()) + ['Mean'],
                             index = [method])
                cb_score = [np.mean(metrics['Cross-Boundary Direction Correctness (A->B)'][x])
                            for x in metrics['Cross-Boundary Direction Correctness (A->B)'].keys()]
                tab.loc[method,:] = cb_score + [np.mean(cb_score)]
                tab.to_csv(save_dir + dataset + 'training_paramsLR005' + str(i) + '_CBDC_scores.csv')
                metrics = utv.evaluate(adata, ground_truth, 'clusters', 'Velocity')
                if exists(save_dir + dataset + 'training_paramsLR005' + str(i) + '_ICC_scores.csv'):
                    tab = pd.read_csv(save_dir + dataset + 'training_paramsLR005' + str(i) + '_ICC_scores.csv', index_col = 0)
                else:
                    tab = pd.DataFrame(columns = list(np.unique(np.concatenate(ground_truth))) + ['Mean'],
                             index = [method])
                icc_score = [np.mean(metrics['In-cluster Coherence'][x]) for x in np.unique(np.concatenate(ground_truth))]
                tab.loc[method,:] = icc_score + [np.mean(icc_score)]
                tab.to_csv(save_dir + dataset + 'training_paramsLR005' + str(i) + '_ICC_scores.csv')
                fix, ax = plt.subplots(1, 1, figsize = (8, 6))
                scv.pl.velocity_embedding_stream(adata, basis='umap', save = False, vkey='Velocity',
                                                 show = False, ax = ax)
                plt.savefig(save_dir + 'UMAPs/' + dataset + 'training_paramsLR005' + str(i) + '_UMAP_' + method + '.svg')

Global seed set to 0


(Running UniTVelo 0.2.5)
2023-05-03 14:05:44
0
1
2
Keeping at most 100000 cells per cluster
Filtered out 19641 genes that are detected 10 counts (shared).
Extracted 2000 highly variable genes.
Leiden clustering ...
Number of Leiden Clusters: 13
Maximal Number of Modules: 14


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 121/200:  60%|████████████████████████████████████████████████████████████████████████████████████████████▍                                                             | 120/200 [04:45<03:10,  2.39s/it, v_num=1, elbo_train=7.56e+6]

ValueError: Expected parameter loc (Tensor of shape ()) of distribution Normal(loc: nan, scale: nan) to satisfy the constraint Real(), but found invalid values:
nan